In [2]:
# 신경망 학습시키기
import scipy.special
import numpy as np

# 신경망 질의
# 신경망 클래스 정의3
class NeuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate):
        # 입력, 은닉, 출력계층
        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        # 가중치 행렬
        self.wih = np.random.normal(0.0, pow(self.hnodes1, - 0.5), (self.hnodes1, self.inodes))
        # 은닉1->은닉2 가중치
        self.whh = np.random.normal(0.0, pow(self.hnodes1, - 0.5), (self.hnodes2, self.hnodes1))
        self.who = np.random.normal(0.0, pow(self.hnodes2, - 0.5), (self.onodes, self.hnodes2))
        
        # 학습률
        self.lr = learningrate
        
        # 활성함수 - 시그모이드
        self.activation_function = lambda x: scipy.special.expit(x)
        
    # 신경망 학습
    def train(self, inputs_list, targets_list):
        # 입력리스트를 2차원 행렬로 변환
        inputs = np.array(inputs_list, ndmin=2).T
        targets = np.array(targets_list, ndmin=2).T
        
        # 1. 입력->은닉1
        # 은닉계층1으로 들어오는 신호를 계산
        hidden_inputs1 = np.dot(self.wih, inputs)
        # 은닉계층1에서 나가는 신호를 계산
        hidden_outputs1 = self.activation_function(hidden_inputs1)
        
        # 2. 은닉1->은닉2
        # 은닉계층2으로 들어오는 신호를 계산
        hidden_inputs2 = np.dot(self.whh, hidden_outputs1)
        # 은닉계층2에서 나가는 신호를 계산
        hidden_outputs2 = self.activation_function(hidden_inputs2)
        
        # 3. 은닉2->출력
        # 최종출력계층으로 들어오는 신호 계산
        final_inputs = np.dot(self.who, hidden_outputs2)
        # 최종출력계층에서 들어오는 신호 계산
        final_outputs = self.activation_function(final_inputs)
        
        # 출력 계층의 오차는 (실제값 - 계산값)
        output_errors = targets - final_outputs
        # 은닉계층과 출력계층 간의 가중치 업데이트
        hidden_errors2 = np.dot(self.who.T, output_errors)
        # 은닉계층2와 은닉계층1의 가중치 업데이트
        hidden_errors1 = np.dot(self.whh.T, hidden_errors2)
        
        # 은닉계층과 출력계층 간의 가중치 업데이트
        self.who += self.lr * np.dot((output_errors * final_outputs * (1.0 - final_outputs)), np.transpose(hidden_outputs2))
        
        # 은닉계층1과 은닉계층2 간의 가중치 업데이트
        self.whh += self.lr * np.dot((hidden_errors2 * hidden_outputs2 * (1.0 - hidden_outputs2)), np.transpose(hidden_outputs1))
        
        # 입력계층과 은닉계층 간의 가중치 업데이트
        self.wih += self.lr * np.dot((hidden_errors1 * hidden_outputs1 * (1.0 - hidden_outputs1)), np.transpose(inputs))
    
    # 신경망 질의
    def query(self, inputs_list):
        # 입력리스트를 2차원 행렬로 변환
        inputs = np.array(inputs_list, ndmin=2).T
        
        # 은닉계층1으로 들어오는 신호를 계산
        hidden_inputs1 = np.dot(self.wih, inputs)
        # 은닉계층1에서 나가는 신호를 계산
        hiddne_outputs1 = self.activation_function(hidden_inputs1)
        
        # 은닉계층2으로 들어오는 신호를 계산
        hidden_inputs2 = np.dot(self.whh, hiddne_outputs1)
        # 은닉계층2에서 나가는 신호를 계산
        hiddne_outputs2 = self.activation_function(hidden_inputs2)
        
        # 최종출력계층으로 들어오는 신호를 계산
        final_inputs = np.dot(self.who, hiddne_outputs2)
        # 최종출력계층으로 나가는 신호를 계산
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs

In [2]:
    
# 노드의 갯수
input_nodes = 784
hidden_nodes1 = 280
hidden_nodes2 = 80
output_nodes = 10
# 학습률
learning_rate = 0.01
# 반복회수
epochs = 7

n = NeuralNetwork(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

# 60,000개의 레코드 학습데이터 모음
# 10,000개의 레코드를 가지는 전체 테스트 데이터 모음
training_data_file = open("mnist_dataset/mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

# 주기를 통해 신경망 학습
# 신경망 학습시키기
for e in range(epochs):
    print(f"epochs {e} done")
    for record in training_data_list:
        # 레코드를 쉼표에 의해 분리
        all_values = record.split(",")
        # 입력 값의 범위와 값 조정
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # 결과 값 생성(실제 값인 0.99 외에는 모두 0.01)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0]은 이 레코드에 대한 결과값
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)


    
# 신경망의 성능 지표가 되는 성적표를 초기화함
scorecard = []

test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

for record in test_data_list:
    # 레코드를 쉼표에 의해 분리
    all_values = record.split(",")
    # 정답은 첫번째 값
    correct_label = int(all_values[0])
    #print(correct_label, "correct label")
    # 입력 값의 범위와 값 조정
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # 신경망에 질의
    outputs = n.query(inputs)
    # 가장 높은 값의 인덱스는 레이블의 인덱스와 일치
    label = np.argmax(outputs)
    #print(label, "network's answer")
    # 정답 또는 오답을 리스트에 추가
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)

# 정답
scorecard_array = np.asarray(scorecard)
# print("performance =", scorecard_array.sum() / scorecard_array.size)
print("hidden_nodes1 =", hidden_nodes1, end="  ")
print("hidden_nodes2 =", hidden_nodes2, end="  ")
print("rate =", learning_rate, end="  ")
print("epochs =", epochs, end="  ")
print("performance =", scorecard_array.sum() / scorecard_array.size)

epochs 0 done
epochs 1 done
epochs 2 done
epochs 3 done
epochs 4 done
epochs 5 done
epochs 6 done
hidden_nodes1 = 280  hidden_nodes2 = 80  rate = 0.01  epochs = 7  performance = 0.9739


In [3]:
# 노드의 갯수
input_nodes = 784
hidden_nodes1 = 500
hidden_nodes2 = 200
output_nodes = 10
# 학습률
learning_rate = 0.01
# 반복회수
epochs = 7

n = NeuralNetwork(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

# 60,000개의 레코드 학습데이터 모음
# 10,000개의 레코드를 가지는 전체 테스트 데이터 모음
training_data_file = open("mnist_dataset/mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

# 주기를 통해 신경망 학습
# 신경망 학습시키기
for e in range(epochs):
    print(f"epochs {e} done")
    for record in training_data_list:
        # 레코드를 쉼표에 의해 분리
        all_values = record.split(",")
        # 입력 값의 범위와 값 조정
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # 결과 값 생성(실제 값인 0.99 외에는 모두 0.01)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0]은 이 레코드에 대한 결과값
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)


    
# 신경망의 성능 지표가 되는 성적표를 초기화함
scorecard = []

test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

for record in test_data_list:
    # 레코드를 쉼표에 의해 분리
    all_values = record.split(",")
    # 정답은 첫번째 값
    correct_label = int(all_values[0])
    #print(correct_label, "correct label")
    # 입력 값의 범위와 값 조정
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # 신경망에 질의
    outputs = n.query(inputs)
    # 가장 높은 값의 인덱스는 레이블의 인덱스와 일치
    label = np.argmax(outputs)
    #print(label, "network's answer")
    # 정답 또는 오답을 리스트에 추가
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)

# 정답
scorecard_array = np.asarray(scorecard)
# print("performance =", scorecard_array.sum() / scorecard_array.size)
print("hidden_nodes1 =", hidden_nodes1, end="  ")
print("hidden_nodes2 =", hidden_nodes2, end="  ")
print("rate =", learning_rate, end="  ")
print("epochs =", epochs, end="  ")
print("performance =", scorecard_array.sum() / scorecard_array.size)

epochs 0 done
epochs 1 done
epochs 2 done
epochs 3 done
epochs 4 done
epochs 5 done
epochs 6 done
hidden_nodes1 = 500  hidden_nodes2 = 200  rate = 0.01  epochs = 7  performance = 0.9747


In [1]:
test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

test_data_list[0]

'7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,84,185,159,151,60,36,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,222,254,254,254,254,241,198,198,198,198,198,198,198,198,170,52,0,0,0,0,0,0,0,0,0,0,0,0,67,114,72,114,163,227,254,225,254,254,254,250,229,254,254,140,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,17,66,14,67,67,67,59,21,236,254,106,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,83,253,209,18,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,22,233,255,83,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,129,254,238,44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,59,249,254,62,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,133,254,187,5,0,

In [3]:
# 노드의 갯수
input_nodes = 784
hidden_nodes1 = 280
hidden_nodes2 = 80
output_nodes = 10
# 학습률
learning_rate = 0.01
# 반복회수
epochs = 10

n = NeuralNetwork(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

# 60,000개의 레코드 학습데이터 모음
# 10,000개의 레코드를 가지는 전체 테스트 데이터 모음
training_data_file = open("mnist_dataset/mnist_train.csv", "r")
training_data_list = training_data_file.readlines()
training_data_file.close()

# 주기를 통해 신경망 학습
# 신경망 학습시키기
for e in range(epochs):
    print(f"epochs {e} done")
    for record in training_data_list:
        # 레코드를 쉼표에 의해 분리
        all_values = record.split(",")
        # 입력 값의 범위와 값 조정
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # 결과 값 생성(실제 값인 0.99 외에는 모두 0.01)
        targets = np.zeros(output_nodes) + 0.01
        # all_values[0]은 이 레코드에 대한 결과값
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)


    
# 신경망의 성능 지표가 되는 성적표를 초기화함
scorecard = []

test_data_file = open("mnist_dataset/mnist_test.csv", "r")
test_data_list = test_data_file.readlines()
test_data_file.close()

for record in test_data_list:
    # 레코드를 쉼표에 의해 분리
    all_values = record.split(",")
    # 정답은 첫번째 값
    correct_label = int(all_values[0])
    #print(correct_label, "correct label")
    # 입력 값의 범위와 값 조정
    inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # 신경망에 질의
    outputs = n.query(inputs)
    # 가장 높은 값의 인덱스는 레이블의 인덱스와 일치
    label = np.argmax(outputs)
    #print(label, "network's answer")
    # 정답 또는 오답을 리스트에 추가
    if (label == correct_label):
        scorecard.append(1)
    else:
        scorecard.append(0)

# 정답
scorecard_array = np.asarray(scorecard)
# print("performance =", scorecard_array.sum() / scorecard_array.size)
print("hidden_nodes1 =", hidden_nodes1, end="  ")
print("hidden_nodes2 =", hidden_nodes2, end="  ")
print("rate =", learning_rate, end="  ")
print("epochs =", epochs, end="  ")
print("performance =", scorecard_array.sum() / scorecard_array.size)

epochs 0 done
epochs 1 done
epochs 2 done
epochs 3 done
epochs 4 done
epochs 5 done
epochs 6 done
epochs 7 done
epochs 8 done
epochs 9 done
hidden_nodes1 = 280  hidden_nodes2 = 80  rate = 0.01  epochs = 10  performance = 0.9763
